# Sanity check para RegretMatching Agent
Este notebook prueba la implementación del RegretMatching Agent en todos los juegos disponibles durante al menos 10 episodios.

In [1]:
import numpy as np
import matplotlib.pyplot as plt

from games.rps import RPS
from games.mp import MP
from games.blotto import Blotto
from games.foraging import Foraging
from agents.regretmatching_t import RegretMatching # Changed import

## Definir Juegos y Configuraciones
Define los juegos a probar y sus configuraciones.

In [2]:
games_to_test = [
    {"name": "RPS", "game": RPS, "config": {}},
    {"name": "MP", "game": MP, "config": {}},
    {"name": "Blotto", "game": Blotto, "config": {"S": 3, "N": 2}},
    {"name": "Foraging", "game": Foraging, "config": {"config": "Foraging-5x5-2p-1f-v3", "seed": 1}},
]

def get_game_instance(game_entry):
    if game_entry["name"] == "Blotto":
        return game_entry["game"](**game_entry["config"])
    elif game_entry["name"] == "Foraging":
        return game_entry["game"](**game_entry["config"])
    else:
        return game_entry["game"]()

## Inicializar RegretMatching para cada juego
Para cada juego, inicializa el RegretMatching para todos los agentes en el entorno.

In [3]:
def create_agents(game):
    # Config básica para RegretMatching Agent
    # Constructor: __init__(self, game, agent, initial=None, seed=None)
    return {
        agent_id: RegretMatching(
            game=game, 
            agent=agent_id, 
            seed=1
        ) for agent_id in game.agents
    }

## Ejecutar episodios para cada juego
Ejecuta al menos 10 episodios para cada juego y recolecta las recompensas acumuladas.

In [4]:
def play_episodes(game, agents, episodes=10):
    recompensas_acumuladas_totales = {agent_id: 0.0 for agent_id in game.agents}
    game_name = game.metadata.get('name', 'UnknownGame') if hasattr(game, 'metadata') and game.metadata else 'UnknownGame'
    
    # Chequeo más robusto para Foraging game, asumiendo que la clase Foraging está importada o identificable
    is_foraging_game = "Foraging" in str(type(game))


    for ep in range(episodes):
        if is_foraging_game:
            print(f"[Foraging Debug] Starting Episode {ep+1}/{episodes}")
        
        # Resetear el game y asegurar que el estado interno de los agentes relacionado con el episodio también se resetee si es necesario
        # game.reset() es el estándar. Los agentes podrían requerir un método específico si trackean datos por episodio más allá de su interacción con el game object.
        # En RegretMatching Agent, la policy y los regrets se acumulan entre episodios, así que no suele ser necesario agent.reset().
        game.reset() 
        
        # Para juegos stateful como Foraging, asegurar que la view del agente del game también se resetee
        # Esto podría implicar resetear caches de observations o estado del game dentro del agente si mantiene algo así
        # Sin embargo, RegretMatching Agent implementado se basa en game.observe() y game.reward(), recalculando la policy según los regrets acumulados, así que no es usual un reset explícito del agente por episodio.

        turn = 0
        # Max turns como safety feature, especialmente para juegos que podrían no terminar correctamente.
        max_turns_per_episode = 200 

        while not (all(game.terminations.values()) or all(game.truncations.values())):
            if is_foraging_game:
                print(f"\n[Foraging Debug] Episode {ep+1}, Turn {turn+1}")
                if hasattr(game, '_last_joint_action_input'):
                    print(f"[Foraging Debug] game._last_joint_action_input before agent actions: {game._last_joint_action_input}")
                else:
                    print("[Foraging Debug] game._last_joint_action_input not available at start of turn.")

            current_actions = {}
            for agent_id in game.agents:
                try:
                    if is_foraging_game:
                        print(f"[Foraging Debug] Agent {agent_id} attempting to choose action...")
                    # El método action del agente debería usar su current policy, actualizada según regrets previos.
                    action = agents[agent_id].action()
                    current_actions[agent_id] = action
                    if is_foraging_game:
                        print(f"[Foraging Debug] Agent {agent_id} chose action: {action}")
                except Exception as e:
                    if is_foraging_game:
                        print(f"[Foraging Debug] ERROR in agent {agent_id}.action(): {e}")
                    # Suele ser mejor imprimir el full traceback para debugging
                    import traceback
                    traceback.print_exc()
                    raise 

            if is_foraging_game:
                print(f"[Foraging Debug] current_actions to be passed to game.step(): {current_actions}")
            
            try:
                # Guardar las acciones para Foraging game para que los agentes las usen en el next turn para calcular regrets
                if is_foraging_game:
                    # Esto asume que el game object puede almacenar esto. Un wrapper podría ser más limpio.
                    game._last_joint_action_input = current_actions.copy() 
                
                game.step(current_actions)

            except Exception as e:
                if is_foraging_game:
                    print(f"[Foraging Debug] ERROR in game.step({current_actions}): {e}")
                import traceback
                traceback.print_exc()
                raise 

            for agent_id in game.agents:
                # Acumular las rewards de este episodio
                recompensas_acumuladas_totales[agent_id] += game.reward(agent_id)
            
            if is_foraging_game:
                print(f"[Foraging Debug] Rewards after turn {turn+1}: { {ag: game.reward(ag) for ag in game.agents} }")
                print(f"[Foraging Debug] Terminations: {game.terminations}")
                print(f"[Foraging Debug] Truncations: {game.truncations}")

            turn += 1
            if turn >= max_turns_per_episode : 
                if is_foraging_game:
                    print(f"[Foraging Debug] Safety break: Exceeded {max_turns_per_episode} turns in episode {ep+1}.")
                # Si se alcanza max_turns, considerar truncation para todos los agentes y así terminar el episodio
                for agent_id in game.agents:
                    game.truncations[agent_id] = True 
                break
        
        if is_foraging_game:
            print(f"[Foraging Debug] Episode {ep+1} finished. Total turns: {turn}")
            print(f"[Foraging Debug] Final cumulative rewards for episode: { {k:v for k,v in recompensas_acumuladas_totales.items()} }")


    return recompensas_acumuladas_totales

## Mostrar resultados
Muestra las recompensas acumuladas para cada agente en cada juego después de 10 episodios.

In [5]:
resultados = {}
for entry in games_to_test:
    print(f"\nProbando {entry['name']} con RegretMatching...")
    game = get_game_instance(entry)
    agents = create_agents(game)
    recompensas = play_episodes(game, agents, episodes=10)
    resultados[entry['name']] = recompensas
    for agent, recompensa in recompensas.items():
        print(f"Agente {agent}: Recompensa total en 10 episodios: {recompensa}")


Probando RPS con RegretMatching...
Agente agent_0: Recompensa total en 10 episodios: -2.0
Agente agent_1: Recompensa total en 10 episodios: 2.0

Probando MP con RegretMatching...
Agente agent_0: Recompensa total en 10 episodios: -2.0
Agente agent_1: Recompensa total en 10 episodios: 2.0

Probando Blotto con RegretMatching...
Agente agent_0: Recompensa total en 10 episodios: 0.0
Agente agent_1: Recompensa total en 10 episodios: 0.0

Probando Foraging con RegretMatching...
[Foraging Debug] Starting Episode 1/10

[Foraging Debug] Episode 1, Turn 1
[Foraging Debug] game._last_joint_action_input not available at start of turn.
[Foraging Debug] Agent agent_0 attempting to choose action...
[Foraging Debug] Agent agent_0 chose action: 5
[Foraging Debug] Agent agent_1 attempting to choose action...
[Foraging Debug] Agent agent_1 chose action: 4
[Foraging Debug] current_actions to be passed to game.step(): {'agent_0': 5, 'agent_1': 4}
[Foraging Debug] Rewards after turn 1: {'agent_0': 0, 'agent

/opt/homebrew/Caskroom/miniconda/base/envs/py311/lib/python3.11/site-packages/gymnasium/utils/passive_env_checker.py:245: UserWarning: WARN: The reward returned by `step()` must be a float, int, np.integer or np.floating, actual type: <class 'list'>
  logger.warn(
